## Imports

In [12]:
import re
import os
import csv
import pandas as pd

In [13]:
from scripts import cleaner
from scripts import entro

In [14]:
from bs4 import BeautifulSoup
import urllib.request

## Data loading

In [15]:
decades = ['1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
decades_dict = {}

#### ignore

~~#### Read src csv lists to df's~~

In [ ]:
# # Takes in one of the decade CSV show lists and returns it as a formatted dataframe
# def csvToDataframe(filename):
#     df = pd.read_csv(filename, header=None, names=['title', 'start', 'end', 'wiki_url', 'transcript_url'])
#     return df

In [ ]:
# for decade in decades:
#     path = "src/" + decade[:-1] + "_shows.csv"
#     decades_dict[decade] = csvToDataframe(path)
#     # print(path + " has been processed")

In [ ]:
# shows = []
# for decade in decades:
#     curr_df = decades_dict[decade]
#     for i, row in curr_df.iterrows():
#         shows.append(row['title'])

#### Read metadata csv's to df's

In [20]:
meta = {}

In [24]:
for decade in decades:
    path = "metadata/episodes_" + decade[:-1] + ".csv"
    meta[decade] = pd.read_csv(path, header=0)

#### Group df's by title and season

In [25]:
for decade in decades:
    curr_df = meta[decade]
    meta[decade] = curr_df.groupby(['title', 'season'])

In [26]:
meta['1950s']['episode'].count()

title        season
I Love Lucy  1         36
             2         32
             3         31
             4         30
             5         26
             6         27
Name: episode, dtype: int64

In [27]:
meta['2000s']['episode'].count()

title                  season
30 Rock                1         21
                       2         15
                       3         22
                       4         21
                       5         22
                       6         21
                       7         12
Big Bang Theory        1         17
                       2         23
                       3         23
                       4         24
                       5         24
                       6         24
                       7         24
                       8         24
                       9         24
                       10        24
                       11        24
                       12        24
Community              1         25
                       2         24
                       3         22
                       4         13
                       5         13
                       6         13
Malcolm in the Middle  1         16
                       2         2

## Retrieving transcripts

In [28]:
a_mark = "{START OF EPISODE "
z_mark = "{END OF EPISODE "
base = "https://www.springfieldspringfield.co.uk/"

In [29]:
# Helper function for getTV()
def getScript(url):
    # Returns list of lines from episode transcript
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    text = soup.find("div", {"class":"scrolling-script-container"}).get_text()
    cleaned = cleaner.clean(text)
    return cleaned
    # return cleaner.split_into_sentences(cleaned)

In [38]:
# Pass in decade as a string value (the key for the current decade we're scraping)
def getTV(decade):
    for grp_idx, grp in meta[decade]:
        t_raw, season, ended = grp_idx[0], grp_idx[1], False
        if season < 7:
            t_clean = re.sub(r' ', '_', t_raw.lower())
            f_path = "transcripts/" + decade + "/" + t_clean + "_season" + str(season) + ".txt"
            # open file to begin writing
            f = open(f_path, "w")
            print("Currently scraping... " + t_raw + ", Season " + str(season))
            # loop through each episode of the current season
            for row_idx, row in grp.iterrows():
                # add episode start marker
                f.write(a_mark + str(row['episode']) + "}" + "\n")
                # get transcript
                text = getScript(base + row['url'])
                lines = cleaner.split_into_sentences(text)
                # add each line to the file
                for i in range(len(lines)):
                    f.write(lines[i])
                    if i < len(lines) - 1:
                        f.write("\n")
                # add episode end marker
                f.write("\n" + z_mark + str(row['episode']) + "}")
            # close the file !!!
            f.close()
            print("Finished scraping... " + t_raw + ", Season " + str(season))
        else:
            if not ended:
                print("Yeah I'm not scraping past 6 seasons bud... sorry.") #SixSeasonsAndAMovie
                ended = True

In [37]:
getTV('1950s')

Currently scraping... I Love Lucy, Season 1
Finished scraping... I Love Lucy, Season 1
Currently scraping... I Love Lucy, Season 2
Finished scraping... I Love Lucy, Season 2
Currently scraping... I Love Lucy, Season 3
Finished scraping... I Love Lucy, Season 3
Currently scraping... I Love Lucy, Season 4
Finished scraping... I Love Lucy, Season 4
Currently scraping... I Love Lucy, Season 5
Finished scraping... I Love Lucy, Season 5
Currently scraping... I Love Lucy, Season 6
Finished scraping... I Love Lucy, Season 6


In [39]:
getTV('1970s')

Currently scraping... M*A*S*H, Season 1
Finished scraping... M*A*S*H, Season 1
Currently scraping... M*A*S*H, Season 2
Finished scraping... M*A*S*H, Season 2
Currently scraping... M*A*S*H, Season 3
Finished scraping... M*A*S*H, Season 3
Currently scraping... M*A*S*H, Season 4
Finished scraping... M*A*S*H, Season 4
Currently scraping... M*A*S*H, Season 5
Finished scraping... M*A*S*H, Season 5
Currently scraping... M*A*S*H, Season 6
Finished scraping... M*A*S*H, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Mork and Mindy, Season 1
Finished scraping... Mork and Mindy, Season 1
Currently scraping... Mork and Mindy, Season 2
Finished scraping... Mork and Mindy, Season 2
Currently scraping... Mork and Mindy, Season 3
Finished scraping... Mork and Mindy, S

## Processing transcripts

In [8]:
pattern = re.compile(r'\{START OF EPISODE \d{2}\}(.+?)\{END OF EPISODE \d{2}\}', re.DOTALL)